In [ ]:
import pandas as pd
import os
import collections

## 1. Calculate the statistic of each record

### Read traffic flow data of each site - 974 sites

In [2]:
def read_df(filename):
    site_df = pd.read_csv('20211213_SCOOT_data/' + filename)
    
    return site_df

In [3]:
def read_df_2(site_df):
    site_dup = site_df.loc[site_df['count'] > 1]
    site_dup['startDate'] = min(site_df['lastUpdate'])
    site_dup['errorPercentage(%)'] = (site_dup['count'].sum() - len(site_dup))/site_df['count'].sum() * 100
    site_dup['allCount'] = site_df['count'].sum()
       
    return site_dup

### Extract the essential information from raw traffic flow data 

In [4]:
def df_clean(site_dup):
    #clean and rearrange df
    new_cols = ['siteSiteid', 'timestamp', 'count', 'errorPercentage(%)', 'startDate']
    site_clean = site_dup[new_cols]

    #extract a date column from timestamp
    timestamp = site_clean['timestamp'].str.split('T', n = 1, expand = True)
    site_clean['date'] = timestamp[0]
    site_clean['time'] = timestamp[1]
    
    timestamp_raw = site_df['timestamp'].str.split('T', n = 1, expand = True)
    site_df['date'] = timestamp_raw[0]
    #display(site_df)
    
    return site_clean

### Get number of consecutive interpolation each day

In [5]:
def get_count_each_day(site_clean):
    #set date & add count
    dates = list(set(list(site_clean['date'])))
    #print(len(dates))

    site_dateCount = {}
    for a in dates:
        b = site_clean.loc[site_clean.date == a , 'count']
        c = site_df.loc[site_df.date == a, 'count']
        site_dateCount[a] = [sum(b), sum(c)]
    #print(site_dateCount)

    dic_site_dateCount = {}
    date = []
    dateCount = []
    dateCountAll = []
    for key, value in site_dateCount.items():
        date.append(key)
        dateCount.append(value[0])
        dateCountAll.append(value[1])
        

    dic_site_dateCount['errorDate'] = date
    dic_site_dateCount['reCount/Day'] = dateCount
    dic_site_dateCount['allCount/Day'] = dateCountAll

    df_site_dateCount = pd.DataFrame.from_dict(dic_site_dateCount)
    df_site_dateCount = df_site_dateCount.sort_values(by=['errorDate'])
    
    return df_site_dateCount

### Get the time of interpolation each day

In [6]:
def get_alltime_each_day(site_clean):
    #set date & merge time
    dates = list(set(list(site_clean['date'])))
    #print(len(dates))

    site_time = {}
    for a in dates:
        b = site_clean.loc[site_clean.date == a , 'time']
        site_time[a] = sorted(list(set(list(b))))
    #print(site_time)

    dic_site_time = {}
    date = []
    time = []
    for key, value in site_time.items():
        date.append(key)
        time.append(value)

    dic_site_time['errorDate'] = date
    dic_site_time['reTime/Day'] = time

    df_site_time = pd.DataFrame.from_dict(dic_site_time)
    df_site_time = df_site_time.sort_values(by=['errorDate'])
    
    return df_site_time

### Merge the two columns together

In [7]:
def merge_to_each_day(df_site_dateCount, df_site_time):
    site_date_time = pd.merge(df_site_dateCount, df_site_time, how="left", on=["errorDate"])
    site_date_time['siteId'] = site_clean['siteSiteid'].tolist()[0]
    site_date_time['errorPercentage(%)'] = site_dup['errorPercentage(%)'].tolist()[0]
    site_date_time['startDate'] = site_dup['startDate'].tolist()[0]
    site_date_time['errorPercentage/Day(%)'] = (site_date_time['reCount/Day'] - 1) / site_date_time['allCount/Day'] * 100
    site_date_time['allCount'] = site_dup['allCount'].tolist()[0]
    new_cols = ['siteId', 'errorDate', 'reCount/Day', 'reTime/Day', 'errorPercentage/Day(%)', 'errorPercentage(%)', 'allCount', 'startDate']
    site_date_time = site_date_time[new_cols]
    
    return site_date_time

### Merge each site in a single raw

In [46]:
def merge_to_each_site(site_date_time):
    # columns: siteId, errorPercentage, countAll, startDate
    site_id = site_date_time['siteId'].tolist()[0]
    error_percentage = site_date_time['errorPercentage(%)'].tolist()[0]
    count_all = site_date_time['allCount'].tolist()[0]
    start_date = site_date_time['startDate'].tolist()[0]
    
    data = [[site_id, error_percentage, count_all, start_date]]
    
    # Create the pandas DataFrame
    each_site = pd.DataFrame(data, columns = ['siteId', 'errorPercentage(%)', 'countAll', 'startDate'])
    
    return each_site

## Implement

In [ ]:
directory = os.fsencode('20211213_SCOOT_data')

all_sites = []
all_sites_date = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename.endswith(".csv"): 
        #print(os.path.join('20211213_SCOOT_data', filename))
        
        site_df = read_df(filename)
        site_dup = read_df_2(site_df)
        site_clean = df_clean(site_dup)
        df_site_dateCount = get_count_each_day(site_clean)
        df_site_time = get_alltime_each_day(site_clean)
        site_date_time = merge_to_each_day(df_site_dateCount, df_site_time)
        
        site_date_time.to_csv('SCOOT_data_statistic/Each_site_statistic/' + filename, index = False)       
        each_site = merge_to_each_site(site_date_time)
        
        all_sites.append(each_site)
        all_sites_date.append(site_date_time)


all_statistic = pd.concat(all_sites)
all_statistic = all_statistic.reset_index(drop=True)

all_sites_day_statistic = pd.concat(all_sites_date)
all_sites_day_statistic = all_sites_day_statistic.reset_index(drop=True)

## 2. Clean the records via statistics

### Get counts 

In [5]:
counts = list(set(list(all_statistic['countAll'])))
#print(len(counts))

counts_sta = {}
for a in counts:
    b = all_statistic.loc[all_statistic.countAll == a , 'countAll']
    c = all_statistic.loc[all_statistic.countAll == a , 'siteId']
    counts_sta[a] = [len(b), list(c)]

collections.OrderedDict(sorted(counts_sta.items()))

dic_counts = {}
counts = []
countNumber = []
sites = []
for key, value in counts_sta.items():
    counts.append(key)
    countNumber.append(value[0])
    sites.append(value[1])


dic_counts['countAll'] = counts
dic_counts['countNum'] = countNumber
dic_counts['sites'] = sites

df_counts = pd.DataFrame.from_dict(dic_counts)
df_counts = df_counts.sort_values(by=['countAll'])
df_counts = df_counts.reset_index(drop=True)

df_counts

,countAll,countNum,sites
0,25181,4,"[GE2291_V, GE2291_W, GE2291_S, GE2291_R]"
1,38810,1,[GD0551_X]
2,79285,8,"[GG2601_T, GG2601_R, GA4771_V, GJ011A_V, GG030..."
3,79286,7,"[GG2601_V, GL142A_A, GB0351_T, GB0351_S, GG260..."
4,79287,33,"[DD1701_A, DD1701_B, DD1701_C, GD237A_R, GD237..."
...,...,...,...
85,79411,10,"[GC2381_R, GJ275A_V, GF0301_D, GF0301_C, GF030..."
86,79412,8,"[GC2381_T, GC2381_S, GJ3301_C, GH020A_A, GJ330..."
87,79413,6,"[GJ3551_V, GJ3551_T, GC0321_A, GC0321_D, GC032..."
88,79414,13,"[GB1301_X, GK0351_R, GJ3551_W, GK0351_V, GH100..."


### Get start date

In [6]:
dates = list(set(list(all_statistic['startDate'])))
#print(len(counts))

counts_sta = {}
for a in dates:
    b = all_statistic.loc[all_statistic.startDate == a , 'startDate']
    c = all_statistic.loc[all_statistic.startDate == a , 'siteId']
    counts_sta[a] = [len(b), list(c)]

#collections.OrderedDict(sorted(counts_sta.items()))

dic_dates = {}
dates = []
dateNumber = []
sites = []
for key, value in counts_sta.items():
    dates.append(key)
    dateNumber.append(value[0])
    sites.append(value[1])


dic_dates['startDate'] = dates
dic_dates['dateNumber'] = dateNumber
dic_dates['sites'] = sites

df_dates = pd.DataFrame.from_dict(dic_dates)
df_dates = df_dates.sort_values(by=['startDate'])
df_dates = df_dates.reset_index(drop=True)

df_dates

,startDate,dateNumber,sites
0,2019-08-05T00:00:01.5814024Z,946,"[GD207A_R, GB0051_W, GJ0131_B, GC237A_B, GH090..."
1,2020-07-11T12:45:00.8072021Z,6,"[GG2151_T, GD210A_B, GA1901_N, GD260A_A, GH262..."
2,2020-07-11T21:30:00.3252876Z,5,"[GA5251_S, GJ0141_X, DD1701_C, GC2381_T, GC238..."
3,2020-07-11T23:00:01.05235Z,1,[GB0401_R]
4,2020-07-22T23:00:01.1726122Z,1,[GC237A_A]
5,2020-07-25T23:00:01.0411601Z,3,"[GH2621_D, GC242A_S, GC2401_C]"
6,2020-08-02T02:45:00.9860526Z,1,[GD260A_B]
7,2020-08-02T04:45:01.0164309Z,4,"[GA2401_A, DD1701_B, GC2411_P, GC2401_B]"
8,2020-08-06T11:00:00.3221056Z,2,"[GH090A_T, DD1701_A]"
9,2020-11-03T14:00:02.4188479Z,1,[GD0551_X]


### Get the error percentage

In [7]:
df_error = all_statistic.loc[all_statistic['errorPercentage(%)'] > 1].iloc[:,:2]
df_error[21:]

,siteId,errorPercentage(%)
21,GJ250B_T,77.889149
22,GG3001_C,68.533629
23,GD3051_A,68.476207
24,GC2381_T,56.706795
25,GC2381_S,56.704276
26,GC2381_R,56.703731
27,GC2411_P,56.695849
28,GC2411_M,56.692953
29,GC2411_N,56.692953
30,GC242A_R,56.667465


### Get the error sites based on above statistic: 
    countsAll < 79000
    startDate != 2019-08-05
    errorPercentage > 1%

In [8]:
count_error = df_counts.loc[df_counts.countAll < 79000 , 'sites'].tolist()
date_error = df_dates.loc[df_dates.startDate != '2019-08-05T00:00:01.5814024Z' , 'sites'].tolist()
error_error = df_error.siteId.tolist()

error_list = []
for i in range(len(count_error)):
    error_list = error_list + count_error[i]
for j in range(len(date_error)):
    error_list = error_list + date_error[j]
error_list = error_list + error_error

In [10]:
error_sites = list(set(error_list))
error_sites_df = pd.DataFrame({'errorSites':error_sites})
error_sites_df

,errorSites
0,GD3051_A
1,GC2401_C
2,GC2401_B
3,GH2621_C
4,GJ0141_X
5,GA1901_N
6,GB0051_W
7,GD0551_X
8,DD1701_C
9,GC242A_S


### Get the error dates:
    errorPercentage/Day(%) > 10

In [20]:
error_index = []
for error_site in error_sites_df.errorSites.tolist():
    #print(error_site)
    all_useful_sites = all_sites_day_statistic.loc[all_sites_day_statistic['siteId'] == error_site]
    error_index = error_index + all_useful_sites.index.tolist()

all_useful_sites = all_sites_day_statistic.drop(error_index)
all_useful_sites = all_useful_sites.sort_values(by=['errorPercentage/Day(%)', 'siteId'])
all_useful_sites = all_useful_sites.reset_index(drop=True)

error_dates = list(set(all_useful_sites.loc[all_useful_sites['errorPercentage/Day(%)'] > 10 , 'errorDate'].tolist()))
error_dates_df = pd.DataFrame(columns=['errorDates'], data=error_dates)
error_dates_df = error_dates_df.sort_values(by=['errorDates'])
error_dates_df = error_dates_df.reset_index(drop=True)
error_dates_df

,errorDates
0,2020-03-12
1,2020-05-05
2,2020-06-24
3,2020-09-25
4,2020-10-30
5,2020-11-17
6,2020-12-02
7,2020-12-05
8,2021-02-23
9,2021-03-02


### Clean the error sites - 926 sites

In [ ]:
#a sample list of raw recorded sites
raw_df = pd.read_csv('20211111_real_time_data_OSGB36.csv')

error_index = []
for error_site in error_sites_df.errorSites.tolist():
    #print(error_site)
    all_error_sites = raw_df.loc[raw_df['siteSiteid'] == error_site]
    error_index = error_index + all_error_sites.index.tolist()

all_useful_sites = raw_df.drop(error_index)
all_useful_sites = all_useful_sites.sort_values(by=['siteSiteid'])
all_useful_sites = all_useful_sites.reset_index(drop=True).iloc[:, 7:]

detector_926 = all_useful_sites[['siteSiteid', 'originDescription', 'originLat', 'originLong', 'originEasting', 'originNorthing']]
detector_926 = detector_926.rename(columns={'siteSiteid': 'siteId','originDescription': 'description', 'originLat': 'lat', 'originLong': 'lon', 'originEasting': 'easting', 'originNorthing': 'northing'})
detector_926['type'] = 'detector'

## 3. Clean the recordes via spatial location - 917 sites

In [ ]:
#a list of distance between recorded sites and closest road
distance_df = pd.read_csv('../SCOOT_data_points-shp/Map matching/closest_road_to_detector_100.csv')
distance_df_15 = distance_df.loc[distance_df.NEAR_DIST < 15]

detector_926['IN_FID'] = detector_926.index
detector_917 = pd.merge(distance_df_15, detector_926, how="left", on=["IN_FID"]).iloc[:,-7:]

## 4. Clean the dataset via consecutive zero 

### Clean the error dates

In [77]:
def data_clean(data, error_dates_df):

    data['date'] = data['lastUpdate'].str.split('T').str[0]
    for i in error_dates_df.errorDates.tolist():
        data = data.drop(data.loc[data['date']==i].index)
    
    data2 = data.sort_values(by=['lastUpdate'])
    data2 = data2.reset_index(drop=True)
    
    return data2

### Delete all zero from original data

In [6]:
def drop_zero_flow(data2):
    data2 = data2[data2['flow'] != 0]
    
    data3 = data2.sort_values(by=['lastUpdate'])
    data3 = data3.reset_index(drop=True)

    return data3

### Check the consecutive dates after drop all the zero

In [6]:
def get_check_df(data3): 
    
    date_range = data3.iloc[[0, -1]].date.tolist()
    start_date = date_range[0]
    end_date = date_range[1]
    full_dates = pd.DataFrame(pd.date_range(start_date, end_date), columns=['date']) #get the list of dates
    #practical_dates = pd.DataFrame({'date':list(set(data3.date))})
    zero_check_df = pd.DataFrame({'siteId': data3.siteSiteid[0],
            'start_date': start_date,
            'end_date': end_date,
            'full_dates': len(full_dates),
            'practical_dates': len(set(data3.date))}, index=[0])

    return zero_check_df

### Implement: Select the sites with the most consecutive flow - 530 sites

In [ ]:
df_list = []
for i in detector_917.siteId.tolist():
    print(i)
    site_df = pd.read_csv('20211213_SCOOT_data/' + i + '&2019_08_05-2021_12_12.csv')
    data2 = data_clean(site_df, error_dates)
    data3 = drop_zero_flow(data2)
    each_check = get_check_df(data3)
    df_list.append(each_check)
    
df_check_consecutive = pd.concat(df_list)
df_check_consecutive = df_check_consecutive.sort_values(by=['siteId']).reset_index(drop=True)
df_check_consecutive

In [ ]:
df_check_consecutive['consecutive_percentage'] = df_check_consecutive['practical_dates']/df_check_consecutive['full_dates']
#select the 530 most useful sites
detector_530 = df_check_consecutive.loc[df_check_consecutive.start_date == '2019-08-05'].loc[df_check_consecutive.practical_dates == 815].sort_values(by=['siteId']).reset_index(drop=True)
detector_530 = pd.merge(detector_530, detector_917, how="left", on='siteId')